In [1]:
# import the necessary packages
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import PIL
import os

In [2]:
def plt_imshow(title, image):
    # convert the image frame BGR to RGB color space and display it
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.title(title)
    plt.grid(False)
    plt.show()

In [3]:
class Config:
    # define the base path to the input dataset and then use it to derive
    # the path to the images directory and annotation CSV file
    BASE_PATH = "C:/Users/HP/DF2009CM/capstone_project/dataset"
    IMAGES_PATH = os.path.sep.join([BASE_PATH, "images"])
    ANNOTS_PATH = os.path.sep.join([BASE_PATH, "annotations"])

    # define the path to the base output directory
    BASE_OUTPUT = "output"

    # define the path to the output serialized model, model training plot,
    # and testing image filenames
    MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "detector.h5"])
    LB_PATH = os.path.sep.join([BASE_OUTPUT, "lb.pickle"])
    PLOT_PATH = os.path.sep.join([BASE_OUTPUT, "plots"])
    TEST_PATHS = os.path.sep.join([BASE_OUTPUT, "test_paths.txt"])

    # initialize our initial learning rate, number of epochs to train
    # for, and the batch size
    INIT_LR = 1e-3
    NUM_EPOCHS = 20
    BATCH_SIZE = 32

# instantiate the config class
config = Config()

In [4]:
# load the contents of the CSV annotations file
print("[INFO] loading dataset...")

# initialize the list of data (images), our target output predictions
# (bounding box coordinates), along with the filenames of the
# individual images

data = []
labels = []
bboxes = []
imagePaths = []

[INFO] loading dataset...


In [5]:
# loop over all CSV files in the annotations directory
for csvPath in paths.list_files(config.ANNOTS_PATH, validExts=(".csv")):
    # load the contents of the current CSV annotations file
    rows = open(csvPath).read().strip().split("\n")
    # loop over the rows
    for row in rows:
        # break the row into the filename, bounding box coordinates,
        # and class label
        row = row.split(",")
        (filename, startX, startY, endX, endY, label) = row
        
        # derive the path to the input image, load the image (in
        # OpenCV format), and grab its dimensions
        imagePath = os.path.sep.join([config.IMAGES_PATH, label,
            filename])
        image = cv2.imread(imagePath)
        (h, w) = image.shape[:2]
        # scale the bounding box coordinates relative to the spatial
        # dimensions of the input image
        startX = float(startX) / w
        startY = float(startY) / h
        endX = float(endX) / w
        endY = float(endY) / h
        
        # load the image and preprocess it
        image = cv2.imread(imagePath)
        image = cv2.resize(image,(224,224))
        image = np.array(image)
        # update our list of data, class labels, bounding boxes, and
        # image paths
        data.append(image.astype('float32'))
        labels.append(label)
        bboxes.append((startX, startY, endX, endY))
        imagePaths.append(imagePath)
        
data = np.asarray(data)
        
# convert the data, class labels, bounding boxes, and image paths to
# NumPy arrays, scaling the input pixel intensities from the range
# [0, 255] to [0, 1]
data /= 255.0
labels = np.array(labels)
bboxes = np.array(bboxes, dtype="float32")
imagePaths = np.array(imagePaths)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
# only there are only two labels in the dataset, then we need to use
# Keras/TensorFlow's utility function as well
if len(lb.classes_) == 2:
    labels = to_categorical(labels)
        

In [6]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
split = train_test_split(data, labels, bboxes, imagePaths,
    test_size=0.20, random_state=42)
# unpack the data split
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]
(trainBBoxes, testBBoxes) = split[4:6]
(trainPaths, testPaths) = split[6:]
# write the testing image paths to disk so that we can use then
# when evaluating/testing our object detector
print("[INFO] saving testing image paths...")
f = open(config.TEST_PATHS, "w")
f.write("\n".join(testPaths))
f.close()

[INFO] saving testing image paths...


In [7]:
# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(224, 224, 3)))
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)

In [8]:
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid",
    name="bounding_box")(bboxHead)
# construct a second fully-connected layer head, this one to predict
# the class label
softmaxHead = Dense(512, activation="relu")(flatten)
softmaxHead = Dropout(0.3)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.3)(softmaxHead)
softmaxHead = Dense(len(lb.classes_), activation="softmax",
    name="class_label")(softmaxHead)

# put together our model which accept an input image and then output
# bounding box coordinates and a class label
model = Model(
    inputs=vgg.input,
    outputs=(bboxHead, softmaxHead))
# define a dictionary to set the loss methods -- categorical
# cross-entropy for the class label head and mean absolute error
# for the bounding box head
losses = {
    "class_label": "categorical_crossentropy",
    "bounding_box": "mean_squared_error",
}
# define a dictionary that specifies the weights per loss (both the
# class label and bounding box outputs will receive equal weight)
lossWeights = {
    "class_label": 1.0,
    "bounding_box": 1.0
}
# initialize the optimizer, compile the model, and show the model
# summary
opt = Adam(learning_rate=config.INIT_LR)
model.compile(loss=losses, optimizer=opt, metrics=["accuracy"], loss_weights=lossWeights)
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 224, 224, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv2 (Conv2D)           (None, 224, 224, 64) 36928       block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_pool (MaxPooling2D)      (None, 112, 112, 64) 0           block1_conv2[0][0]               
______________________________________________________________________________________________

In [9]:
# construct a dictionary for our target training outputs
trainTargets = {
    "class_label": trainLabels,
    "bounding_box": trainBBoxes
}
# construct a second dictionary, this one for our target testing
# outputs
testTargets = {
    "class_label": testLabels,
    "bounding_box": testBBoxes
}

# train the network for bounding box regression and class label
# prediction
print("[INFO] training model...")
H = model.fit(
    trainImages, trainTargets,
    validation_data=(testImages, testTargets),
    batch_size=config.BATCH_SIZE,
    epochs=config.NUM_EPOCHS,
    verbose=1)
# serialize the model to disk
print("[INFO] saving object detector model...")
model.save(config.MODEL_PATH, save_format="h5")
# serialize the label binarizer to disk
print("[INFO] saving label binarizer...")
f = open(config.LB_PATH, "wb")
f.write(pickle.dumps(lb))
f.close()

[INFO] training model...
Epoch 1/20
14/14 [==============================] - 48s 3s/step - loss: 3.5617 - bounding_box_loss: 0.0602 - class_label_loss: 3.5015 - bounding_box_accuracy: 0.4378 - class_label_accuracy: 0.3134 - val_loss: 0.9375 - val_bounding_box_loss: 0.0431 - val_class_label_loss: 0.8944 - val_bounding_box_accuracy: 0.5780 - val_class_label_accuracy: 0.6055
Epoch 2/20
14/14 [==============================] - 47s 3s/step - loss: 1.1543 - bounding_box_loss: 0.0354 - class_label_loss: 1.1189 - bounding_box_accuracy: 0.5737 - class_label_accuracy: 0.5622 - val_loss: 0.8453 - val_bounding_box_loss: 0.0362 - val_class_label_loss: 0.8091 - val_bounding_box_accuracy: 0.6881 - val_class_label_accuracy: 0.6422
Epoch 3/20
14/14 [==============================] - 48s 3s/step - loss: 0.6744 - bounding_box_loss: 0.0211 - class_label_loss: 0.6533 - bounding_box_accuracy: 0.6267 - class_label_accuracy: 0.7212 - val_loss: 0.7828 - val_bounding_box_loss: 0.0285 - val_class_label_loss: 0.7

In [10]:
# plot the total loss, label loss, and bounding box loss
lossNames = ["loss", "class_label_loss", "bounding_box_loss"]
N = np.arange(0, config.NUM_EPOCHS)
plt.style.use("ggplot")
(fig, ax) = plt.subplots(3, 1, figsize=(13, 13))
# loop over the loss names
for (i, l) in enumerate(lossNames):
    # plot the loss for both the training and validation data
    title = "Loss for {}".format(l) if l != "loss" else "Total loss"
    ax[i].set_title(title)
    ax[i].set_xlabel("Epoch #")
    ax[i].set_ylabel("Loss")
    ax[i].plot(N, H.history[l], label=l)
    ax[i].plot(N, H.history["val_" + l], label="val_" + l)
    ax[i].legend()
# save the losses figure and create a new figure for the accuracies
plt.tight_layout()
plotPath = os.path.sep.join([config.PLOT_PATH, "losses.png"])
plt.savefig(plotPath)
plt.close()

## data augmentation

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import re
import random
from tqdm import tqdm
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
#Creating a function to create images

def create_images(name,save_path,image_path):
    #Generating a random number for rotation
    rotation = random.randint(-20,20)
    #Creating Image Data Generator:
    augmentation = ImageDataGenerator(rotation_range=rotation,horizontal_flip=True)
    
    #Reading the image from given path.
    image_org = cv2.imread(image_path+'/'+name)
    
    #Saving original image to folder.
    #image_org.save(save_path+'/'+name)
    
    #Getting the numpy array of image.
    image_arr = np.array(image_org)
    #image_arr = image.img_to_array(image_org)
    
    #Expanding dimensions of image array
    image_arr = image_arr.reshape((1,) + image_arr.shape)
    
    #Using Flow to generate 2 new images for single image
    for arr,val in zip(augmentation.flow(image_arr, batch_size=1, save_format='jpeg'),range(1)):
        name = name.split('.')[0]
        img_save = image.array_to_img(arr[0], scale=False)
        img_save.save(save_path+'/z_'+name+'_aug_'+str(val)+'.jpeg')
    #return paths

In [ ]:
scratches_list = os.listdir('C:/Users/HP/DF2009CM/capstone_project/dataset/images/Scratches')
scratches_aug = "dataset/aug_images/Scratches"

for i in tqdm(scratches_list):
    create_images(i,scratches_aug,'C:/Users/HP/DF2009CM/capstone_project/dataset/images/Scratches')

In [ ]:
scratches_list = os.listdir('C:/Users/HP/DF2009CM/capstone_project/dataset/images/Dents')
scratches_aug = "dataset/aug_images/Dents"

for i in tqdm(scratches_list):
    create_images(i,scratches_aug,'C:/Users/HP/DF2009CM/capstone_project/dataset/images/Dents')